# CellLabeller Tutorial: Cell Type Label Transfer

This tutorial demonstrates how to use CellLabeller to transfer cell type labels from a reference single-cell dataset to a query dataset using scVI integration and XGBoost classification.

## Overview
The pipeline consists of the following steps:
1. Load reference and query datasets
2. Subset datasets to common genes
3. Integrate datasets with scVI (≥200 epochs)
4. Select features (genes and/or scVI latent space)
5. Tune XGBoost hyperparameters (GPU and CPU)
6. Train and evaluate models
7. Make predictions on query data

## 1. Import Required Libraries

In [ ]:
import sys
import os
from pathlib import Path

# Add the repository to path if needed
# sys.path.insert(0, '/path/to/CellLabeller')

import numpy as np
import pandas as pd
import anndata as ad
import scanpy as sc
import scvi
import xgboost as xgb
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import warnings
warnings.filterwarnings('ignore')

# Set visualization parameters
sc.set_figure_params(dpi=100, figsize=(6, 6))
sns.set_style('whitegrid')

print("✓ Libraries imported successfully")

## 2. Load Reference and Query Datasets

Load your AnnData objects. You can load from various formats:
- `.h5ad` files
- `.h5` files
- `.csv` files (with metadata)
- `.loom` files

In [ ]:
# MODIFY THESE PATHS TO YOUR DATA
REFERENCE_DATA_PATH = "path/to/reference_data.h5ad"  # Replace with your reference data
QUERY_DATA_PATH = "path/to/query_data.h5ad"  # Replace with your query data
RESULTS_DIR = "./celllabeller_results"

# Load datasets
print("Loading reference dataset...")
adata_ref = sc.read_h5ad(REFERENCE_DATA_PATH)

print("Loading query dataset...")
adata_query = sc.read_h5ad(QUERY_DATA_PATH)

print(f"\nReference dataset shape: {adata_ref.shape}")
print(f"Query dataset shape: {adata_query.shape}")

# Display cell type distribution in reference
CELL_TYPE_KEY = "cell_type"  # Modify if your column name is different
if CELL_TYPE_KEY in adata_ref.obs:
    print(f"\nReference cell types:")
    print(adata_ref.obs[CELL_TYPE_KEY].value_counts())
    print(f"\nTotal cell types: {adata_ref.obs[CELL_TYPE_KEY].nunique()}")
else:
    print(f"Warning: {CELL_TYPE_KEY} not found in reference metadata")

## 3. Subset Datasets with Common Genes

Ensure both datasets have the same genes for fair comparison.

In [ ]:
# Find common genes
ref_genes = set(adata_ref.var_names)
query_genes = set(adata_query.var_names)
common_genes = sorted(list(ref_genes.intersection(query_genes)))

print(f"Reference genes: {len(ref_genes)}")
print(f"Query genes: {len(query_genes)}")
print(f"Common genes: {len(common_genes)}")
print(f"Reference genes removed: {len(ref_genes) - len(common_genes)}")
print(f"Query genes removed: {len(query_genes) - len(common_genes)}")

# Subset to common genes
adata_ref = adata_ref[:, common_genes]
adata_query = adata_query[:, common_genes]

print(f"\nNew reference shape: {adata_ref.shape}")
print(f"New query shape: {adata_query.shape}")

## 4. Integrate Datasets with scVI

Concatenate reference and query datasets and integrate using scVI (minimum 200 epochs).

In [ ]:
# Add dataset identifier
adata_ref.obs["dataset"] = "reference"
adata_query.obs["dataset"] = "query"

# Concatenate datasets
print("Concatenating datasets...")
adata_integrated = ad.concat(
    [adata_ref, adata_query],
    axis=0,
    label="dataset_id",
    keys=["reference", "query"],
)

print(f"Integrated dataset shape: {adata_integrated.shape}")

# Preprocessing
print("\nPreprocessing data...")
sc.pp.normalize_total(adata_integrated, target_sum=1e4)
sc.pp.log1p(adata_integrated)

print("✓ Preprocessing complete")

In [ ]:
# Setup and train scVI model
print("Setting up scVI model...")
scvi.model.SCVI.setup_anndata(
    adata_integrated,
    batch_key="dataset",
)

# Train scVI (minimum 200 epochs)
N_EPOCHS = 250  # At least 200 as required
print(f"\nTraining scVI model for {N_EPOCHS} epochs...")
print("(This may take a few minutes depending on dataset size)\n")

model_scvi = scvi.model.SCVI(adata_integrated)
model_scvi.train(
    max_epochs=N_EPOCHS,
    early_stopping=True,
    early_stopping_patience=10,
    verbose=False,
)

print("✓ scVI training complete")

In [ ]:
# Extract latent representation
print("Extracting scVI latent representation...")
adata_integrated.obsm["X_scvi"] = model_scvi.get_latent_representation()

print(f"Latent space shape: {adata_integrated.obsm['X_scvi'].shape}")

# Save scVI model
os.makedirs(RESULTS_DIR, exist_ok=True)
scvi_model_dir = os.path.join(RESULTS_DIR, "scvi_model")
print(f"Saving scVI model to {scvi_model_dir}...")
model_scvi.save(scvi_model_dir, overwrite=True)

print("✓ scVI latent representation extracted and model saved")

## 5. Feature Engineering and Selection

Select features based on user preference:
- `genes`: Top genes selected by f-score
- `scvi_latent`: scVI latent space representation
- `combined`: Both genes and latent space

In [ ]:
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.preprocessing import StandardScaler, LabelEncoder

# Feature selection parameters
N_GENES = 500  # Number of top genes to select
FEATURE_TYPE = "combined"  # Choose: "genes", "scvi_latent", or "combined"

print(f"\nFeature Selection: {FEATURE_TYPE}")
print("="*50)

# Get reference data for feature selection
ref_mask = adata_integrated.obs["dataset"] == "reference"
X_ref = adata_integrated.X[ref_mask]
y_ref = adata_integrated.obs.loc[ref_mask, CELL_TYPE_KEY].values

print(f"Reference samples for feature selection: {ref_mask.sum()}")

# Select top genes
print(f"\nSelecting top {N_GENES} genes...")
selector = SelectKBest(f_classif, k=N_GENES)
X_genes_selected = selector.fit_transform(X_ref, y_ref)
selected_genes = adata_integrated.var_names[selector.get_support(indices=True)].tolist()

print(f"Selected genes: {len(selected_genes)}")

# Get gene features for full dataset
X_genes = adata_integrated[:, selected_genes].X
if hasattr(X_genes, 'toarray'):
    X_genes = X_genes.toarray()

print(f"Gene feature matrix shape: {X_genes.shape}")

# Get scVI latent features
X_scvi = adata_integrated.obsm["X_scvi"]
print(f"scVI latent matrix shape: {X_scvi.shape}")

# Combine features based on user choice
if FEATURE_TYPE == "genes":
    X_features = X_genes
    feature_names = selected_genes
elif FEATURE_TYPE == "scvi_latent":
    X_features = X_scvi
    feature_names = [f"scvi_{i}" for i in range(X_scvi.shape[1])]
elif FEATURE_TYPE == "combined":
    X_features = np.hstack([X_genes, X_scvi])
    feature_names = selected_genes + [f"scvi_{i}" for i in range(X_scvi.shape[1])]

# Standardize features
print(f"\nStandardizing {len(feature_names)} features...")
scaler = StandardScaler()
X_features = scaler.fit_transform(X_features)

print(f"Final feature matrix shape: {X_features.shape}")
print(f"Feature types: {FEATURE_TYPE}")

## 6. Hyperparameter Testing with XGBoost

Conduct hyperparameter tuning using Bayesian optimization (Optuna) for both GPU and CPU.

In [ ]:
import optuna
from optuna.samplers import TPESampler
from sklearn.model_selection import train_test_split, cross_val_score

# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(adata_integrated.obs[CELL_TYPE_KEY].values)

print(f"Number of cell types: {len(label_encoder.classes_)}")
print(f"Cell types: {label_encoder.classes_}")

# Split data (80% train, 20% test from reference data)
X_train, X_test, y_train, y_test = train_test_split(
    X_features[ref_mask],
    y_encoded[ref_mask],
    test_size=0.2,
    random_state=42,
    stratify=y_encoded[ref_mask],
)

print(f"\nTrain set: {X_train.shape[0]} samples")
print(f"Test set: {X_test.shape[0]} samples")

In [ ]:
# Hyperparameter tuning
N_TRIALS = 30  # Number of optimization trials (more = longer but better results)

def objective_gpu_true(trial):
    """Objective function for GPU=True"""
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 1000),
        "max_depth": trial.suggest_int("max_depth", 3, 12),
        "learning_rate": trial.suggest_float("learning_rate", 1e-4, 1e-1, log=True),
        "subsample": trial.suggest_float("subsample", 0.6, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0),
        "gamma": trial.suggest_float("gamma", 0, 10),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-5, 10, log=True),
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-5, 10, log=True),
    }
    
    try:
        clf = xgb.XGBClassifier(
            **params,
            tree_method="gpu_hist",
            gpu_id=0,
            objective="multi:softmax" if len(label_encoder.classes_) > 2 else "binary:logistic",
            eval_metric="mlogloss" if len(label_encoder.classes_) > 2 else "logloss",
            num_class=len(label_encoder.classes_) if len(label_encoder.classes_) > 2 else None,
            random_state=42,
        )
        scores = cross_val_score(clf, X_train, y_train, cv=5, scoring="balanced_accuracy")
        return scores.mean()
    except Exception as e:
        print(f"GPU trial failed: {e}")
        return float("-inf")

def objective_gpu_false(trial):
    """Objective function for GPU=False"""
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 1000),
        "max_depth": trial.suggest_int("max_depth", 3, 12),
        "learning_rate": trial.suggest_float("learning_rate", 1e-4, 1e-1, log=True),
        "subsample": trial.suggest_float("subsample", 0.6, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0),
        "gamma": trial.suggest_float("gamma", 0, 10),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-5, 10, log=True),
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-5, 10, log=True),
    }
    
    try:
        clf = xgb.XGBClassifier(
            **params,
            tree_method="hist",
            objective="multi:softmax" if len(label_encoder.classes_) > 2 else "binary:logistic",
            eval_metric="mlogloss" if len(label_encoder.classes_) > 2 else "logloss",
            num_class=len(label_encoder.classes_) if len(label_encoder.classes_) > 2 else None,
            random_state=42,
        )
        scores = cross_val_score(clf, X_train, y_train, cv=5, scoring="balanced_accuracy")
        return scores.mean()
    except Exception as e:
        print(f"CPU trial failed: {e}")
        return float("-inf")

print("Starting hyperparameter tuning...")
print("="*60)

In [ ]:
# Try GPU tuning (may fail if GPU not available)
print("\n[1/2] Tuning with GPU=True...")
try:
    sampler_gpu = TPESampler(seed=42)
    study_gpu = optuna.create_study(direction="maximize", sampler=sampler_gpu)
    study_gpu.optimize(objective_gpu_true, n_trials=N_TRIALS, show_progress_bar=True)
    best_params_gpu = study_gpu.best_params
    best_score_gpu = study_gpu.best_value
    print(f"✓ GPU tuning complete. Best score: {best_score_gpu:.4f}")
except Exception as e:
    print(f"⚠ GPU tuning failed: {e}")
    print("  Proceeding with CPU only...")
    best_params_gpu = None
    best_score_gpu = None

In [ ]:
# CPU tuning
print("\n[2/2] Tuning with GPU=False (CPU)...")
sampler_cpu = TPESampler(seed=42)
study_cpu = optuna.create_study(direction="maximize", sampler=sampler_cpu)
study_cpu.optimize(objective_gpu_false, n_trials=N_TRIALS, show_progress_bar=True)
best_params_cpu = study_cpu.best_params
best_score_cpu = study_cpu.best_value
print(f"✓ CPU tuning complete. Best score: {best_score_cpu:.4f}")

print("\n" + "="*60)
print("Hyperparameter tuning complete!")

In [ ]:
# Display best parameters
print("\nBest Parameters:")
print("-"*60)
if best_params_gpu is not None:
    print(f"GPU=True (Score: {best_score_gpu:.4f}):")
    for param, value in best_params_gpu.items():
        print(f"  {param}: {value}")
else:
    print("GPU=True: Not available")

print(f"\nGPU=False (Score: {best_score_cpu:.4f}):")
for param, value in best_params_cpu.items():
    print(f"  {param}: {value}")

## 7. Model Training and Evaluation

Train final XGBoost models using the optimized hyperparameters.

In [ ]:
from sklearn.metrics import (
    accuracy_score, balanced_accuracy_score, f1_score,
    confusion_matrix, classification_report
)

def train_model_with_params(params, use_gpu=False):
    """Train XGBoost model with given parameters"""
    training_params = params.copy()
    training_params.update({
        "tree_method": "gpu_hist" if use_gpu else "hist",
        "gpu_id": 0 if use_gpu else -1,
        "objective": "multi:softmax" if len(label_encoder.classes_) > 2 else "binary:logistic",
        "eval_metric": "mlogloss" if len(label_encoder.classes_) > 2 else "logloss",
        "num_class": len(label_encoder.classes_) if len(label_encoder.classes_) > 2 else None,
        "random_state": 42,
    })
    
    # Remove None values
    training_params = {k: v for k, v in training_params.items() if v is not None}
    
    model = xgb.XGBClassifier(**training_params)
    model.fit(
        X_train, y_train,
        eval_set=[(X_test, y_test)],
        early_stopping_rounds=10,
        verbose=False,
    )
    return model

print("Training final models...\n")

# Train CPU model (always available)
print("[1/2] Training with CPU...")
model_cpu = train_model_with_params(best_params_cpu, use_gpu=False)
print("✓ CPU model trained")

# Train GPU model if parameters available
if best_params_gpu is not None:
    print("[2/2] Training with GPU...")
    try:
        model_gpu = train_model_with_params(best_params_gpu, use_gpu=True)
        print("✓ GPU model trained")
    except Exception as e:
        print(f"⚠ GPU model training failed: {e}")
        model_gpu = None
else:
    model_gpu = None
    print("[2/2] Skipping GPU model (tuning not completed)")

## 8. Score Predictions on Training and Test Data

In [ ]:
def evaluate_model(model, X_tr, y_tr, X_te, y_te, device_name="Model"):
    """Evaluate model on training and test sets"""
    y_train_pred = model.predict(X_tr)
    y_test_pred = model.predict(X_te)
    
    train_acc = accuracy_score(y_tr, y_train_pred)
    test_acc = accuracy_score(y_te, y_test_pred)
    
    train_bal_acc = balanced_accuracy_score(y_tr, y_train_pred)
    test_bal_acc = balanced_accuracy_score(y_te, y_test_pred)
    
    train_f1 = f1_score(y_tr, y_train_pred, average="weighted", zero_division=0)
    test_f1 = f1_score(y_te, y_test_pred, average="weighted", zero_division=0)
    
    print(f"\n{device_name} Performance:")
    print(f"  Train Accuracy: {train_acc:.4f}")
    print(f"  Test Accuracy:  {test_acc:.4f}")
    print(f"  Train Balanced Accuracy: {train_bal_acc:.4f}")
    print(f"  Test Balanced Accuracy:  {test_bal_acc:.4f}")
    print(f"  Train F1 (weighted):     {train_f1:.4f}")
    print(f"  Test F1 (weighted):      {test_f1:.4f}")
    
    return {
        "device": device_name,
        "train_accuracy": train_acc,
        "test_accuracy": test_acc,
        "train_balanced_accuracy": train_bal_acc,
        "test_balanced_accuracy": test_bal_acc,
        "train_f1": train_f1,
        "test_f1": test_f1,
        "y_test_pred": y_test_pred,
        "y_train_pred": y_train_pred,
    }

print("="*60)
print("MODEL EVALUATION")
print("="*60)

results_cpu = evaluate_model(model_cpu, X_train, y_train, X_test, y_test, device_name="CPU Model")

if model_gpu is not None:
    results_gpu = evaluate_model(model_gpu, X_train, y_train, X_test, y_test, device_name="GPU Model")

In [ ]:
# Detailed classification report
print("\n" + "="*60)
print("DETAILED CLASSIFICATION REPORT (CPU)")
print("="*60)
print(classification_report(
    y_test, results_cpu["y_test_pred"],
    target_names=label_encoder.classes_.tolist(),
    zero_division=0,
))

In [ ]:
# Confusion matrix visualization
fig, axes = plt.subplots(1, 1 if model_gpu is None else 2, figsize=(12 if model_gpu is None else 18, 5))

if model_gpu is None:
    axes = [axes]

# CPU confusion matrix
cm_cpu = confusion_matrix(y_test, results_cpu["y_test_pred"])
sns.heatmap(cm_cpu, annot=True, fmt='d', cmap='Blues', ax=axes[0],
            xticklabels=label_encoder.classes_,
            yticklabels=label_encoder.classes_,
            cbar_kws={'label': 'Count'})
axes[0].set_title('CPU Model - Confusion Matrix (Test Set)')
axes[0].set_xlabel('Predicted')
axes[0].set_ylabel('True')

# GPU confusion matrix (if available)
if model_gpu is not None:
    cm_gpu = confusion_matrix(y_test, results_gpu["y_test_pred"])
    sns.heatmap(cm_gpu, annot=True, fmt='d', cmap='Greens', ax=axes[1],
                xticklabels=label_encoder.classes_,
                yticklabels=label_encoder.classes_,
                cbar_kws={'label': 'Count'})
    axes[1].set_title('GPU Model - Confusion Matrix (Test Set)')
    axes[1].set_xlabel('Predicted')
    axes[1].set_ylabel('True')

plt.tight_layout()
plt.savefig(os.path.join(RESULTS_DIR, 'confusion_matrices.png'), dpi=300, bbox_inches='tight')
plt.show()

print(f"\n✓ Confusion matrix saved to {RESULTS_DIR}/confusion_matrices.png")

## 9. Save Results and Model Artifacts

In [ ]:
import pickle

print("Saving models and results...\n")

# Save CPU model
cpu_model_path = os.path.join(RESULTS_DIR, "xgboost_model_cpu.pkl")
with open(cpu_model_path, "wb") as f:
    pickle.dump(model_cpu, f)
print(f"✓ CPU model saved: {cpu_model_path}")

# Save GPU model if available
if model_gpu is not None:
    gpu_model_path = os.path.join(RESULTS_DIR, "xgboost_model_gpu.pkl")
    with open(gpu_model_path, "wb") as f:
        pickle.dump(model_gpu, f)
    print(f"✓ GPU model saved: {gpu_model_path}")

# Save label encoder
encoder_path = os.path.join(RESULTS_DIR, "label_encoder.pkl")
with open(encoder_path, "wb") as f:
    pickle.dump(label_encoder, f)
print(f"✓ Label encoder saved: {encoder_path}")

# Save results
results_cpu_path = os.path.join(RESULTS_DIR, "evaluation_results_cpu.pkl")
with open(results_cpu_path, "wb") as f:
    pickle.dump(results_cpu, f)
print(f"✓ CPU results saved: {results_cpu_path}")

if model_gpu is not None:
    results_gpu_path = os.path.join(RESULTS_DIR, "evaluation_results_gpu.pkl")
    with open(results_gpu_path, "wb") as f:
        pickle.dump(results_gpu, f)
    print(f"✓ GPU results saved: {results_gpu_path}")

# Save hyperparameter tuning results
tuning_results = {
    "cpu": {"best_params": best_params_cpu, "best_score": best_score_cpu},
    "gpu": {"best_params": best_params_gpu, "best_score": best_score_gpu} if best_params_gpu else None,
}
tuning_path = os.path.join(RESULTS_DIR, "tuning_results.pkl")
with open(tuning_path, "wb") as f:
    pickle.dump(tuning_results, f)
print(f"✓ Tuning results saved: {tuning_path}")

In [ ]:
# Save integrated data
print("\nSaving integrated data...")
integrated_path = os.path.join(RESULTS_DIR, "integrated_data.h5ad")
adata_integrated.write_h5ad(integrated_path)
print(f"✓ Integrated data saved: {integrated_path}")

# Save feature information
feature_info = pd.DataFrame({
    "feature_name": feature_names,
    "feature_type": ["gene"] * len(selected_genes) + ["scvi"] * (len(feature_names) - len(selected_genes)),
})
feature_path = os.path.join(RESULTS_DIR, "feature_info.csv")
feature_info.to_csv(feature_path, index=False)
print(f"✓ Feature information saved: {feature_path}")

print(f"\n✓ All results saved to: {RESULTS_DIR}")

## 10. Predict Cell Types for Query Cells

Use the trained model to predict cell types for the query dataset.

In [ ]:
# Get query features
query_mask = adata_integrated.obs["dataset"] == "query"
X_query = X_features[query_mask]

print(f"Query samples: {query_mask.sum()}")
print(f"Query feature matrix shape: {X_query.shape}")

# Make predictions
print("\nPredicting cell types for query cells...")
y_query_pred_encoded_cpu = model_cpu.predict(X_query)
y_query_pred_cpu = label_encoder.inverse_transform(y_query_pred_encoded_cpu)

print("Predicted cell type distribution:")
pred_counts = pd.Series(y_query_pred_cpu).value_counts()
print(pred_counts)

# Get prediction probabilities
y_query_proba_cpu = model_cpu.predict_proba(X_query)
print(f"\nPrediction probabilities shape: {y_query_proba_cpu.shape}")
print(f"Mean confidence: {y_query_proba_cpu.max(axis=1).mean():.4f}")

In [ ]:
# GPU predictions (if available)
if model_gpu is not None:
    print("\nGPU Predictions:")
    y_query_pred_encoded_gpu = model_gpu.predict(X_query)
    y_query_pred_gpu = label_encoder.inverse_transform(y_query_pred_encoded_gpu)
    
    print("Predicted cell type distribution:")
    pred_counts_gpu = pd.Series(y_query_pred_gpu).value_counts()
    print(pred_counts_gpu)
    
    # Prediction probabilities
    y_query_proba_gpu = model_gpu.predict_proba(X_query)
    print(f"\nMean confidence: {y_query_proba_gpu.max(axis=1).mean():.4f}")

In [ ]:
# Add predictions to query AnnData object
adata_query.obs["predicted_cell_type"] = y_query_pred_cpu
adata_query.obs["prediction_confidence"] = y_query_proba_cpu.max(axis=1)

# Add class-specific probabilities
for i, cell_type in enumerate(label_encoder.classes_):
    adata_query.obs[f"prob_{cell_type}"] = y_query_proba_cpu[:, i]

print("✓ Predictions added to query AnnData object")
print(f"\nColumns in query data: {list(adata_query.obs.columns)}")

In [ ]:
# Save predicted query data
query_pred_path = os.path.join(RESULTS_DIR, "query_with_predictions.h5ad")
adata_query.write_h5ad(query_pred_path)
print(f"✓ Query data with predictions saved: {query_pred_path}")

# Save predictions as CSV
predictions_df = pd.DataFrame({
    "cell_id": adata_query.obs_names,
    "predicted_cell_type": y_query_pred_cpu,
    "confidence": y_query_proba_cpu.max(axis=1),
})
for i, cell_type in enumerate(label_encoder.classes_):
    predictions_df[f"prob_{cell_type}"] = y_query_proba_cpu[:, i]

predictions_csv_path = os.path.join(RESULTS_DIR, "query_predictions.csv")
predictions_df.to_csv(predictions_csv_path, index=False)
print(f"✓ Predictions saved as CSV: {predictions_csv_path}")

## Summary and Visualization

In [ ]:
# Create summary report
summary_report = f"""
╔════════════════════════════════════════════════════════════════╗
║           CELLLABELLER ANALYSIS SUMMARY REPORT                 ║
╚════════════════════════════════════════════════════════════════╝

📊 DATASET INFORMATION
{'─'*62}
  Reference dataset shape:        {adata_ref.shape}
  Query dataset shape:            {adata_query.shape}
  Common genes:                   {len(common_genes)}
  Integration method:             scVI
  scVI epochs:                    {N_EPOCHS}

🔧 FEATURE ENGINEERING
{'─'*62}
  Feature type:                   {FEATURE_TYPE}
  Total features:                 {len(feature_names)}
  Gene features:                  {len(selected_genes)}
  scVI latent components:         {X_scvi.shape[1]}

🎯 MODEL TRAINING
{'─'*62}
  Training samples:               {X_train.shape[0]}
  Test samples:                   {X_test.shape[0]}
  Number of cell types:           {len(label_encoder.classes_)}
  Cell types:                     {', '.join(label_encoder.classes_)}

📈 CPU MODEL PERFORMANCE
{'─'*62}
  Train Accuracy:                 {results_cpu['train_accuracy']:.4f}
  Test Accuracy:                  {results_cpu['test_accuracy']:.4f}
  Train Balanced Accuracy:        {results_cpu['train_balanced_accuracy']:.4f}
  Test Balanced Accuracy:         {results_cpu['test_balanced_accuracy']:.4f}
  Train F1 (weighted):            {results_cpu['train_f1']:.4f}
  Test F1 (weighted):             {results_cpu['test_f1']:.4f}

💾 RESULTS DIRECTORY
{'─'*62}
  Location:                       {RESULTS_DIR}
  Models:                         ✓
  Tuning results:                 ✓
  Integrated data:                ✓
  Predictions:                    ✓

✅ ANALYSIS COMPLETE
"""

print(summary_report)

# Save report
report_path = os.path.join(RESULTS_DIR, "ANALYSIS_REPORT.txt")
with open(report_path, "w") as f:
    f.write(summary_report)
print(f"Report saved to: {report_path}")

In [ ]:
# Visualization of results
fig = plt.figure(figsize=(14, 10))
gs = fig.add_gridspec(3, 2, hspace=0.3, wspace=0.3)

# 1. Reference cell type distribution
ax1 = fig.add_subplot(gs[0, 0])
adata_ref.obs[CELL_TYPE_KEY].value_counts().plot(kind='barh', ax=ax1, color='steelblue')
ax1.set_title('Reference Cell Type Distribution', fontweight='bold')
ax1.set_xlabel('Count')

# 2. Query predicted cell type distribution
ax2 = fig.add_subplot(gs[0, 1])
pd.Series(y_query_pred_cpu).value_counts()[label_encoder.classes_].plot(kind='barh', ax=ax2, color='darkgreen')
ax2.set_title('Query Predicted Cell Type Distribution', fontweight='bold')
ax2.set_xlabel('Count')

# 3. Model performance comparison
ax3 = fig.add_subplot(gs[1, 0])
metrics = ['Train Acc', 'Test Acc', 'Train Bal Acc', 'Test Bal Acc']
values = [
    results_cpu['train_accuracy'],
    results_cpu['test_accuracy'],
    results_cpu['train_balanced_accuracy'],
    results_cpu['test_balanced_accuracy'],
]
ax3.bar(metrics, values, color='steelblue', alpha=0.8)
ax3.set_ylim([0, 1])
ax3.set_ylabel('Score')
ax3.set_title('CPU Model Performance Metrics', fontweight='bold')
ax3.axhline(y=0.5, color='red', linestyle='--', alpha=0.5, label='Random')
for i, v in enumerate(values):
    ax3.text(i, v + 0.02, f'{v:.3f}', ha='center', va='bottom', fontweight='bold')

# 4. Prediction confidence distribution
ax4 = fig.add_subplot(gs[1, 1])
ax4.hist(y_query_proba_cpu.max(axis=1), bins=30, color='darkgreen', alpha=0.7, edgecolor='black')
ax4.set_xlabel('Prediction Confidence')
ax4.set_ylabel('Number of Cells')
ax4.set_title(f'Query Prediction Confidence\n(Mean: {y_query_proba_cpu.max(axis=1).mean():.4f})', fontweight='bold')

# 5. Feature contribution (top 20 genes if available)
ax5 = fig.add_subplot(gs[2, :])
if len(selected_genes) > 0:
    feature_importance = model_cpu.feature_importances_[:len(selected_genes)]
    top_features_idx = np.argsort(feature_importance)[-20:]
    top_genes = np.array(selected_genes)[top_features_idx]
    top_importance = feature_importance[top_features_idx]
    
    y_pos = np.arange(len(top_genes))
    ax5.barh(y_pos, top_importance, color='coral', alpha=0.8)
    ax5.set_yticks(y_pos)
    ax5.set_yticklabels(top_genes, fontsize=9)
    ax5.set_xlabel('Feature Importance')
    ax5.set_title('Top 20 Most Important Genes', fontweight='bold')
    ax5.invert_yaxis()

plt.savefig(os.path.join(RESULTS_DIR, 'analysis_summary.png'), dpi=300, bbox_inches='tight')
plt.show()

print(f"✓ Summary visualization saved to {RESULTS_DIR}/analysis_summary.png")

## Next Steps

Your cell type label transfer analysis is complete! Here's what you can do next:

### 1. **Validate Predictions**
   - Compare predicted labels with any existing annotations in your query data
   - Check prediction confidence scores (look for low-confidence predictions)
   - Investigate cells with conflicting predictions between GPU/CPU models

### 2. **Explore Results**
   - Review confusion matrices to identify which cell types are commonly misclassified
   - Examine feature importance to understand which genes drive the classification
   - Analyze prediction probabilities for uncertain cells

### 3. **Refine the Model**
   - Adjust hyperparameters if needed (increase `N_TRIALS` for more thorough search)
   - Try different feature combinations (`FEATURE_TYPE`)
   - Increase scVI epochs for better integration (`N_EPOCHS`)

### 4. **Use the Trained Model**
   - Load the saved model to predict labels for new query datasets
   - See the CellLabeller documentation for production usage examples

### 5. **Visualization**
   - Create UMAP/t-SNE plots with predicted labels
   - Visualize feature importance and prediction confidence
   - Generate publication-quality figures

### Useful Commands for Loading and Using Models
```python
import pickle
from sklearn.preprocessing import LabelEncoder

# Load saved model and encoder
with open('celllabeller_results/xgboost_model_cpu.pkl', 'rb') as f:
    model = pickle.load(f)

with open('celllabeller_results/label_encoder.pkl', 'rb') as f:
    encoder = pickle.load(f)

# Make predictions on new features
y_pred_encoded = model.predict(new_features)
y_pred_labels = encoder.inverse_transform(y_pred_encoded)
y_proba = model.predict_proba(new_features)
```